<a href="https://colab.research.google.com/github/denniesbor/spwio/blob/from_colab/ghosh.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Connect google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import tqdm
import pickle
import pprint
import string

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# Set the data and working folders
working_dir = "/content/drive/MyDrive/spwio"
data_path = os.path.join(working_dir, "data")

In [78]:
# Read the US industry gross output and technical coefficients
technical_coefficients = pd.read_csv(os.path.join(data_path, "direct_requirements.csv")) # total requirements table

# Real US gross output by industires, in billion dollars
gross_output = pd.read_csv(os.path.join(data_path, "us_gross_output.csv"))

# 2022 US Gross output
industry_output = gross_output["2022Q3"]

print(industry_output)

0      433.2
1      541.8
2      510.6
3     1536.2
4     5545.3
5     2182.5
6     2099.2
7     1373.9
8     2462.8
9     7110.3
10    5049.1
11    3009.1
12    1646.7
13     738.2
14    4049.2
Name: 2022Q3, dtype: float64


## Ghosh Model

Ghosh model estimates how a change in industries input affects the output of the forward linked industrial sectors.
Ghosh model is given by:

$x' = v'G$

where G is

$G = (I-B)^{-1}$

B is derived from technical coefficients and industries gross output by

$B = \hat{X}^{-1} A \hat{X}$

The hat symbol matrix X consist of industral outputs as diagonal elements.

In [79]:
# A matrix
tech_matrix = pd.read_csv(os.path.join(data_path, "technology_matrix.csv"))

tech_matrix.set_index("Commodity Description", inplace=True)
tech_matrix

,"Agriculture, forestry, fishing, and hunting",Mining,Utilities,Construction,Manufacturing,Wholesale trade,Retail trade,Transportation and warehousing,Information,"Finance, insurance, real estate, rental, and leasing",Professional and business services,"Educational services, health care, and social assistance","Arts, entertainment, recreation, accommodation, and food services","Other services, except government",Government
Commodity Description,,,,,,,,,,,,,,,
"Agriculture, forestry, fishing, and hunting",0.255497,0.000233,0.000000,0.001160,5.242680e-02,0.000015,0.000744,0.000015,0.000000,4.000000e-07,0.000648,0.000092,0.007919,0.000088,0.002069
Mining,0.001961,0.102799,0.051607,0.008352,7.763570e-02,0.000037,0.000079,0.000352,0.001477,2.640000e-05,0.000638,0.000674,0.001387,0.001551,0.006488
Utilities,0.016244,0.027625,0.073770,0.006059,1.314060e-02,0.008619,0.025438,0.010318,0.008404,6.522400e-03,0.004047,0.007608,0.022582,0.010199,0.010311
Construction,0.002627,0.006337,0.011229,0.000183,3.301000e-03,0.002162,0.005187,0.004549,0.002141,1.695690e-02,0.001155,0.001495,0.004850,0.004750,0.028244
Manufacturing,0.140615,0.112831,0.036314,0.234560,3.308800e-01,0.051939,0.028988,0.090810,0.041233,1.047120e-02,0.043201,0.081260,0.078761,0.069634,0.136964
Wholesale trade,0.086914,0.026882,0.011544,0.057040,7.397350e-02,0.027169,0.015908,0.018301,0.015476,4.478400e-03,0.011018,0.023499,0.020276,0.016163,0.024826
Retail trade,0.002886,0.001144,0.006139,0.051691,2.883700e-03,0.000295,0.003140,0.011777,0.000159,1.371500e-03,0.001169,0.000258,0.006806,0.009152,0.000067
Transportation and warehousing,0.030743,0.022802,0.040003,0.019245,3.035140e-02,0.055182,0.048313,0.144016,0.009795,6.537000e-03,0.013374,0.009994,0.010611,0.005656,0.020242
Information,0.001009,0.001872,0.013114,0.005011,3.464200e-03,0.009856,0.018957,0.007365,0.129173,8.775600e-03,0.026006,0.009923,0.021636,0.015707,0.038439


In [80]:
# Technology matrix is inclusive of value added and imports which are excluded
A = tech_matrix.iloc[:15].to_numpy()
print(A)

# Identity matrix
I = np.eye(A.shape[1])

[[2.554971e-01 2.329000e-04 0.000000e+00 1.160100e-03 5.242680e-02
  1.490000e-05 7.436000e-04 1.540000e-05 0.000000e+00 4.000000e-07
  6.482000e-04 9.190000e-05 7.919200e-03 8.780000e-05 2.068800e-03]
 [1.961300e-03 1.027994e-01 5.160710e-02 8.352400e-03 7.763570e-02
  3.730000e-05 7.950000e-05 3.516000e-04 1.477100e-03 2.640000e-05
  6.385000e-04 6.745000e-04 1.387400e-03 1.550800e-03 6.488000e-03]
 [1.624430e-02 2.762530e-02 7.377020e-02 6.059300e-03 1.314060e-02
  8.619100e-03 2.543800e-02 1.031800e-02 8.404200e-03 6.522400e-03
  4.047000e-03 7.608400e-03 2.258170e-02 1.019870e-02 1.031090e-02]
 [2.626600e-03 6.336700e-03 1.122850e-02 1.834000e-04 3.301000e-03
  2.161800e-03 5.186700e-03 4.548900e-03 2.140700e-03 1.695690e-02
  1.154800e-03 1.495000e-03 4.849700e-03 4.749800e-03 2.824430e-02]
 [1.406149e-01 1.128311e-01 3.631390e-02 2.345595e-01 3.308800e-01
  5.193890e-02 2.898820e-02 9.081030e-02 4.123260e-02 1.047120e-02
  4.320070e-02 8.125960e-02 7.876070e-02 6.963440e-02 1.36

In [81]:
# Computing B matrix
gross_diag = np.diag(industry_output) # Make the industrial output as the diagonal in a n by n matrix where n is number of industries/commodities
gross_diag_inv = np.linalg.inv(gross_diag) # Inverse of the diagonal matrix of outputs
B = np.dot(np.dot(gross_diag_inv, A), gross_diag) # Get the B coefficient matrix

# Allocation coefficients
print(B[1])

[1.56817121e-03 1.02799400e-01 4.86352626e-02 2.36820910e-02
 7.94598094e-01 1.50253322e-04 3.08022148e-04 8.91589590e-04
 6.71428918e-03 3.46459801e-04 5.95025904e-03 3.74610179e-03
 4.21674341e-03 2.11295784e-03 4.84887590e-02]


In [82]:
# Ghosh model
ghosh_model = np.linalg.inv(I - B)

## Ghosh Model

We shall leverage the Ghosh Model to estimate changes in gross output due to changes in value added.
The GDP calculations are based on the final demand and VA, ignoring double counting in intra and inter-industry gross output. Proceeding in this section, we shall apply relative values to US GDP to estimate, for example, how much losses of the gross production are incurred for a change in value added.

In [83]:
# Load the US GDP by 15 NAICs sectors
us_gdp_df = pd.read_csv(os.path.join(data_path, "gdp_sector_us_22.csv"))
us_gdp_df.set_index("Unnamed: 0", inplace=True)
us_gdp_df

,2021Q1,2021Q2,2021Q3,2021Q4,2022Q1,2022Q2,2022Q3,2022Q4,2023Q1,2023Q2,2023Q3
Unnamed: 0,,,,,,,,,,,
"Agriculture, forestry, fishing, and hunting",187.4,183.4,181.3,183.0,175.0,168.5,167.0,169.7,180.5,182.1,179.3
Mining,287.6,270.6,265.0,254.8,240.1,229.6,244.4,267.1,274.7,292.8,301.3
Utilities,310.9,315.0,319.8,320.7,311.6,324.2,312.1,325.7,323.2,364.0,336.3
Construction,892.0,908.0,888.2,864.3,871.4,833.6,803.1,802.9,809.4,820.3,850.9
Manufacturing,2216.1,2231.2,2231.0,2314.0,2301.0,2262.3,2266.9,2280.9,2227.1,2262.3,2312.9
Wholesale trade,1215.8,1208.3,1170.2,1177.0,1161.2,1134.5,1143.8,1147.8,1141.1,1125.8,1118.8
Retail trade,1312.6,1247.9,1194.1,1210.2,1154.8,1159.9,1188.2,1226.2,1259.9,1251.1,1318.5
Transportation and warehousing,661.3,677.5,695.3,708.3,694.5,693.6,703.7,705.2,715.5,729.0,736.1
Information,1315.4,1383.2,1421.3,1482.2,1473.5,1486.0,1516.9,1545.6,1562.2,1570.9,1625.4


In [67]:
# Convert GDP vals as numpy array
va = us_gdp_df["2022Q3"].to_numpy()
va_transposed = np.transpose(va)

# Sum the US GDP (in billion dollars)
us_gdp_df["2022Q3"].sum()

21952.4

In [70]:
# Apply a 5% impact on GDP
va_transposed_5_perc = 0.05 * va_transposed

# Change in Gross output is v' @ Ghosh
x_delta_5_perc =  va_transposed_5_perc @ ghosh_model

print(f"Change in gross output \n {x_delta_5_perc}")
print("")

# For a 5% on the US value added, the resulting change in gross output is
("Total change in output {}" .format(np.sum(x_delta_5_perc)))

Change in gross output 
 [ 20.5327078   25.13894575  24.98032664  76.8303116  294.32807833
 106.39172268 102.45904259  67.96023396 131.02262554 352.62430629
 257.59313646 152.94540048  81.60316065  34.14442155 210.57723216]



'Total change in output 1939.1316524729536'

A 5% impact on GDP would amount to a loss of 2 T in US gross output.

**In testing the model, a 100% impact would lead to 100% loss in gross output. Therefore, mat multiplying the total US gdp by the ghosh model should be equal to the total gross output**

In [71]:
# A 100% impact/ total economic shutdown
x_delta_100_perc =  va_transposed @ ghosh_model

print(f"Change in gross output, 100% impact: \n {x_delta_100_perc}")
print("")
print(f"2022 US Gross output by sectors: \n {np.asarray(industry_output)}")

Change in gross output, 100% impact: 
 [ 410.65415605  502.7789151   499.60653278 1536.606232   5886.56156651
 2127.83445356 2049.18085174 1359.20467913 2620.45251073 7052.48612579
 5151.86272926 3058.90800965 1632.06321296  682.88843103 4211.54464316]

2022 US Gross output by sectors: 
 [ 433.2  541.8  510.6 1536.2 5545.3 2182.5 2099.2 1373.9 2462.8 7110.3
 5049.1 3009.1 1646.7  738.2 4049.2]


In [72]:
np.sum(x_delta_100_perc) # Change in gross output

38782.633049459066

In [73]:
np.sum(industry_output)

38288.09999999999

## Estimating indirect downstream impacts (Backward Linkages) due to change in value added